In [12]:
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, SpatialDropout1D
from keras.layers import SimpleRNN # new!
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
# output directory name:
output_dir = 'model_output/rnn'

# training:
epochs = 16 # way more!
batch_size = 128

# vector-space embedding:
n_dim = 64
n_unique_words = 10000
max_review_length = 100 # lowered due to vanishing gradient over time
pad_type = trunc_type = 'pre'
drop_embed = 0.2

# RNN layer architecture:
n_rnn = 256
drop_rnn = 0.2

# dense layer architecture:
# n_dense = 256
# dropout = 0.2

In [14]:
(x_train, y_train), (x_valid, y_valid) = imdb.load_data(num_words=n_unique_words) # removed n_words_to_skip

In [15]:
x_train = pad_sequences(x_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
x_valid = pad_sequences(x_valid, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

### RNN

In [16]:
model = Sequential()
model.add(Embedding(n_unique_words, n_dim, input_length=max_review_length))
model.add(SpatialDropout1D(drop_embed))

model.add(SimpleRNN(n_rnn, dropout=drop_rnn))
# model.add(Dense(n_dense, activation='relu')) # typically don't see top dense layer in NLP like in
# model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid'))


In [21]:
model.build(input_shape=(None, max_review_length)) # Build the model explicitly
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 100, 64)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_3             │ (None, 100, 64)        │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ (None, 256)            │        82,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 722,433 (2.76 MB)

 Trainable params: 722,433 (2.76 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.keras")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [23]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_valid, y_valid), callbacks=[modelcheckpoint])

Epoch 1/16
196/196 ━━━━━━━━━━━━━━━━━━━━ 74s 362ms/step - accuracy: 0.5242 - loss: 0.6930 - val_accuracy: 0.5710 - val_loss: 0.6617
Epoch 2/16
196/196 ━━━━━━━━━━━━━━━━━━━━ 77s 392ms/step - accuracy: 0.5962 - loss: 0.6551 - val_accuracy: 0.5996 - val_loss: 0.6516
Epoch 3/16
196/196 ━━━━━━━━━━━━━━━━━━━━ 67s 344ms/step - accuracy: 0.6835 - loss: 0.5828 - val_accuracy: 0.6427 - val_loss: 0.6180
Epoch 4/16
196/196 ━━━━━━━━━━━━━━━━━━━━ 66s 336ms/step - accuracy: 0.7257 - loss: 0.5312 - val_accuracy: 0.6830 - val_loss: 0.6174
Epoch 5/16
196/196 ━━━━━━━━━━━━━━━━━━━━ 64s 326ms/step - accuracy: 0.7909 - loss: 0.4416 - val_accuracy: 0.6700 - val_loss: 0.6158
Epoch 6/16
196/196 ━━━━━━━━━━━━━━━━━━━━ 68s 348ms/step - accuracy: 0.8127 - loss: 0.4124 - val_accuracy: 0.7600 - val_loss: 0.5654
Epoch 7/16
196/196 ━━━━━━━━━━━━━━━━━━━━ 77s 322ms/step - accuracy: 0.8537 - loss: 0.3426 - val_accuracy: 0.7459 - val_loss: 0.5963
Epoch 8/16
196/196 ━━━━━━━━━━━━━━━━━━━━ 63s 322ms/step - accuracy: 0.8714 - loss: 0

In [ ]:
# resultado: bosta

RNNs are only able to backpropagate through ~10 time steps
before the gradient diminishes so much that parameter updates become negligibly small.

They're rarely used in practice. More sophisticated recurrent layer types like LSTMs, which can backpropagate through ~100 time steps, are far more common.